In [6]:
# STEP 1: Upload the three required USDA CSV files
from google.colab import files
uploaded = files.upload()


In [22]:
# STEP 2: Import necessary libraries
import pandas as pd

# STEP 3: Load the CSVs
food_df = pd.read_csv("/content/Untitled Folder/food.csv")
nutrient_df = pd.read_csv("/content/Untitled Folder/nutrient.csv")
food_nutrient_df = pd.read_csv("/content/Untitled Folder/food_nutrient.csv")

# STEP 4: Define the nutrients we care about
target_nutrients = {
    "Energy": "calories",
    "Protein": "protein",
    "Total lipid (fat)": "fat",
    "Carbohydrate, by difference": "carbs",
    "Sugars, total including NLEA": "sugar",
    "Iron, Fe": "iron"
}


# STEP 5: Get nutrient IDs for our nutrients
nutrient_ids = nutrient_df[nutrient_df['name'].isin(target_nutrients.keys())]
nutrient_id_map = dict(zip(nutrient_ids['id'], nutrient_ids['name']))

# STEP 6: Filter food_nutrient to only relevant nutrients
filtered_fns = food_nutrient_df[food_nutrient_df['nutrient_id'].isin(nutrient_id_map.keys())]

# STEP 7: Pivot table to make one row per food with nutrient columns
pivot = filtered_fns.pivot_table(index='fdc_id',
                                 columns='nutrient_id',
                                 values='amount',
                                 aggfunc='first')

# STEP 8: Rename columns to readable names (calories, protein, etc.)
pivot.columns = [target_nutrients[nutrient_id_map[i]] for i in pivot.columns]

# STEP 9: Merge with food names
merged = food_df[['fdc_id', 'description']].merge(pivot, on='fdc_id')

# STEP 10: Drop rows with missing data
cleaned = merged.dropna().reset_index(drop=True)

# STEP 11: Show sample cleaned foods
cleaned


<ipython-input-22-ef001d958c76>:7: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  food_nutrient_df = pd.read_csv("/content/Untitled Folder/food_nutrient.csv")


,fdc_id,description,protein,fat,carbs,calories,calories,iron
0,321358,"Hummus, commercial",7.35,17.10,14.90,229.0,960.0,2.41
1,321359,"Milk, reduced fat, fluid, 2% milkfat, with add...",3.35,1.90,4.91,50.0,209.0,0.00
2,321360,"Tomatoes, grape, raw",0.83,0.63,5.51,27.0,113.0,0.33
3,321611,"Beans, snap, green, canned, regular pack, drai...",1.04,0.39,4.11,21.0,86.0,0.78
4,321900,"Broccoli, raw",2.57,0.34,6.29,32.0,132.0,0.69
...,...,...,...,...,...,...,...,...
125,1105430,"Apples, red delicious, with skin, raw",0.19,0.21,14.80,62.0,258.0,0.00
126,1105547,"Apples, honeycrisp, with skin, raw",0.10,0.10,14.70,60.0,252.0,0.00
127,1105664,"Apples, granny smith, with skin, raw",0.27,0.14,14.20,59.0,247.0,0.07
128,1105781,"Apples, gala, with skin, raw",0.13,0.15,14.80,61.0,256.0,0.06


In [13]:
def clean_name(name):
    return name.replace(", raw", "").replace(", cooked", "").strip()

cleaned["description"] = cleaned["description"].apply(clean_name)


In [23]:
# Save to CSV and download
cleaned.to_csv("cleaned_food_database.csv", index=False)
files.download("cleaned_food_database.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>